# AtomsBase integration

[AtomsBase.jl](https://github.com/JuliaMolSim/AtomsBase.jl) is a common interface
for representing atomic structures in Julia. DFTK directly supports using such
structures to run a calculation as is demonstrated here.

In [1]:
using DFTK

## Feeding an AtomsBase AbstractSystem to DFTK
In this example we construct a silicon system using the `ase.build.bulk` routine
from the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
(ASE), which is exposed by [ASEconvert](https://github.com/mfherbst/ASEconvert.jl)
as an AtomsBase `AbstractSystem`.

In [2]:
# Construct bulk system and convert to an AbstractSystem
using ASEconvert
system_ase = ase.build.bulk("Si")
system = pyconvert(AbstractSystem, system_ase)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0    2.715    2.715;
                            2.715        0    2.715;
                            2.715    2.715        0]u"Å"

    Atom(Si, [       0,        0,        0]u"Å")
    Atom(Si, [  1.3575,   1.3575,   1.3575]u"Å")


To use an AbstractSystem in DFTK, we attach pseudopotentials, construct a DFT model,
discretise and solve:

In [3]:
system = attach_psp(system; Si="hgh/lda/si-q4")

model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921715930380                   -0.69    5.8         
  2   -7.926168000741       -2.35       -1.22    1.0    211ms
  3   -7.926837095280       -3.17       -2.37    1.8    261ms
  4   -7.926861467379       -4.61       -3.02    2.6    247ms
  5   -7.926861638771       -6.77       -3.37    1.9    219ms
  6   -7.926861667351       -7.54       -3.75    1.6    211ms
  7   -7.926861680244       -7.89       -4.31    1.5    232ms
  8   -7.926861681775       -8.81       -4.84    2.0    224ms
  9   -7.926861681843      -10.17       -5.06    1.9    223ms
 10   -7.926861681871      -10.56       -5.81    1.2    203ms
 11   -7.926861681873      -11.70       -6.45    2.0    242ms
 12   -7.926861681873      -13.18       -6.92    2.1    227ms
 13   -7.926861681873      -14.57       -7.78    2.1    222ms
 14   -7.926861681873   +  -15.05       -8.86    3.1    273ms


If we did not want to use ASE we could of course use any other package
which yields an AbstractSystem object. This includes:

### Reading a system using AtomsIO

In [4]:
using AtomsIO

# Read a file using [AtomsIO](https://github.com/mfherbst/AtomsIO.jl),
# which directly yields an AbstractSystem.
system = load_system("Si.extxyz")

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921682937108                   -0.69    5.8         
  2   -7.926160064788       -2.35       -1.22    1.0    265ms
  3   -7.926837606154       -3.17       -2.37    1.8    233ms
  4   -7.926861516278       -4.62       -3.06    2.9    255ms
  5   -7.926861651707       -6.87       -3.46    1.9    220ms
  6   -7.926861672910       -7.67       -3.88    1.9    253ms
  7   -7.926861680303       -8.13       -4.30    1.4    205ms
  8   -7.926861681727       -8.85       -4.81    1.6    214ms
  9   -7.926861681860       -9.88       -5.27    1.8    219ms
 10   -7.926861681871      -10.95       -5.90    2.0    245ms
 11   -7.926861681873      -11.90       -6.83    1.9    230ms
 12   -7.926861681873      -13.51       -7.65    2.9    256ms
 13   -7.926861681873   +  -14.45       -7.96    2.6    255ms
 14   -7.926861681873      -14.57       -8.34    1.5    210ms


The same could be achieved using [ExtXYZ](https://github.com/libAtoms/ExtXYZ.jl)
by `system = Atoms(read_frame("Si.extxyz"))`,
since the `ExtXYZ.Atoms` object is directly AtomsBase-compatible.

### Directly setting up a system in AtomsBase

In [5]:
using AtomsBase
using Unitful
using UnitfulAtomic

# Construct a system in the AtomsBase world
a = 10.26u"bohr"  # Silicon lattice constant
lattice = a / 2 * [[0, 1, 1.],  # Lattice as vector of vectors
                   [1, 0, 1.],
                   [1, 1, 0.]]
atoms  = [:Si => ones(3)/8, :Si => -ones(3)/8]
system = periodic_system(atoms, lattice; fractional=true)

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921657594270                   -0.69    6.0         
  2   -7.926164129976       -2.35       -1.22    1.0    202ms
  3   -7.926840180385       -3.17       -2.37    1.8    290ms
  4   -7.926864921819       -4.61       -3.01    2.9    253ms
  5   -7.926865043366       -6.92       -3.34    1.6    201ms
  6   -7.926865077971       -7.46       -3.73    1.5    197ms
  7   -7.926865091811       -7.86       -4.42    1.4    192ms


## Obtaining an AbstractSystem from DFTK data

At any point we can also get back the DFTK model as an
AtomsBase-compatible `AbstractSystem`:

In [6]:
second_system = atomic_system(model)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0     5.13     5.13;
                             5.13        0     5.13;
                             5.13     5.13        0]u"a₀"

    Atom(Si, [  1.2825,   1.2825,   1.2825]u"a₀")
    Atom(Si, [ -1.2825,  -1.2825,  -1.2825]u"a₀")


Similarly DFTK offers a method to the `atomic_system` and `periodic_system` functions
(from AtomsBase), which enable a seamless conversion of the usual data structures for
setting up DFTK calculations into an `AbstractSystem`:

In [7]:
lattice = 5.431u"Å" / 2 * [[0 1 1.];
                           [1 0 1.];
                           [1 1 0.]];
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

third_system = atomic_system(lattice, atoms, positions)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0  5.13155  5.13155;
                          5.13155        0  5.13155;
                          5.13155  5.13155        0]u"a₀"

    Atom(Si, [ 1.28289,  1.28289,  1.28289]u"a₀")
    Atom(Si, [-1.28289, -1.28289, -1.28289]u"a₀")
